# Robust Statistics and Distribution Analysis

## Overview

Standard z-scores assume normally distributed data, but crime rates are heavily right-skewed
with many zero-count counties and a few extreme outliers. This notebook evaluates several
robust statistical methods to identify genuinely anomalous counties while controlling for
false discoveries.

**Methods compared:**
- Standard z-scores vs. Robust (MAD-based) z-scores
- False Discovery Rate (FDR) correction (Benjamini-Hochberg)
- Empirical Bayes shrinkage for small-county stabilization
- Log-transformed z-scores, Poisson p-values, Negative Binomial p-values, Percentile ranks

**Data:** `county_decade_outliers.csv` -- county-decade level rates with all statistical scores pre-computed.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings("ignore")

plt.rcParams.update({
    "figure.figsize": (14, 6),
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "font.size": 11,
    "axes.grid": True,
    "grid.alpha": 0.3,
})

print("Libraries loaded successfully.")

## 1. Load Data

In [ ]:
DATA_PATH = "../data/analysis/county_decade_outliers.csv"

df = pd.read_csv(DATA_PATH)

print(f"Dataset shape: {df.shape[0]:,} rows x {df.shape[1]} columns")
print(f"\nColumns:\n{df.columns.tolist()}")
print(f"\nDecades present: {sorted(df['decade'].unique())}")
print(f"Unique states:   {df['State'].nunique()}")
print(f"Unique counties: {df['County'].nunique()}")
df.head(3)

In [ ]:
# Quick statistical summary of the key rate columns
rate_cols = ["missing_per_100k", "bodies_per_100k", "mp_z_score", "mp_robust_z",
             "mp_log_z", "mp_poisson_p", "mp_nb_p", "mp_percentile",
             "bodies_percentile", "shrinkage_weight"]
df[rate_cols].describe().round(3)

## 2. Distribution Diagnostics

Crime rates are count-derived and bounded at zero, which almost always produces a right-skewed
distribution. Below we visualize the empirical distributions and check normality with QQ plots.

In [ ]:
# --- Histograms showing right skew ---
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Missing persons rate -- raw
ax = axes[0, 0]
mp_rate = df["missing_per_100k"].dropna()
ax.hist(mp_rate, bins=80, color="steelblue", edgecolor="white", alpha=0.85)
ax.axvline(mp_rate.median(), color="orange", ls="--", lw=2, label=f"Median = {mp_rate.median():.1f}")
ax.axvline(mp_rate.mean(), color="red", ls="-", lw=2, label=f"Mean = {mp_rate.mean():.1f}")
ax.set_title("Missing Persons per 100k (raw)")
ax.set_xlabel("Rate per 100k")
ax.set_ylabel("Count")
ax.legend()

# Missing persons rate -- log scale
ax = axes[0, 1]
mp_pos = mp_rate[mp_rate > 0]
ax.hist(np.log1p(mp_pos), bins=60, color="steelblue", edgecolor="white", alpha=0.85)
ax.set_title("Missing Persons per 100k (log1p transform)")
ax.set_xlabel("log(1 + rate)")
ax.set_ylabel("Count")

# Bodies rate -- raw
ax = axes[1, 0]
bd_rate = df["bodies_per_100k"].dropna()
ax.hist(bd_rate, bins=80, color="darkred", edgecolor="white", alpha=0.85)
ax.axvline(bd_rate.median(), color="orange", ls="--", lw=2, label=f"Median = {bd_rate.median():.1f}")
ax.axvline(bd_rate.mean(), color="red", ls="-", lw=2, label=f"Mean = {bd_rate.mean():.1f}")
ax.set_title("Unidentified Bodies per 100k (raw)")
ax.set_xlabel("Rate per 100k")
ax.set_ylabel("Count")
ax.legend()

# Bodies rate -- log scale
ax = axes[1, 1]
bd_pos = bd_rate[bd_rate > 0]
ax.hist(np.log1p(bd_pos), bins=60, color="darkred", edgecolor="white", alpha=0.85)
ax.set_title("Unidentified Bodies per 100k (log1p transform)")
ax.set_xlabel("log(1 + rate)")
ax.set_ylabel("Count")

fig.suptitle("Distribution of Crime Rates -- Evidence of Right Skew", fontsize=15, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()

# Skewness statistics
print("Skewness (raw):")
print(f"  missing_per_100k : {mp_rate.skew():.2f}")
print(f"  bodies_per_100k  : {bd_rate.skew():.2f}")
print(f"\nSkewness (log1p):")
print(f"  missing_per_100k : {np.log1p(mp_pos).skew():.2f}")
print(f"  bodies_per_100k  : {np.log1p(bd_pos).skew():.2f}")
print("\nSkewness > 1 indicates heavy right skew; log transform reduces it substantially.")

In [ ]:
# --- QQ Plots vs. Normal Distribution ---
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# QQ plot -- missing_per_100k
ax = axes[0]
stats.probplot(mp_rate, dist="norm", plot=ax)
ax.set_title("QQ Plot: missing_per_100k vs Normal")
ax.get_lines()[0].set(color="steelblue", markersize=3, alpha=0.5)
ax.get_lines()[1].set(color="red", linewidth=2)

# QQ plot -- bodies_per_100k
ax = axes[1]
stats.probplot(bd_rate, dist="norm", plot=ax)
ax.set_title("QQ Plot: bodies_per_100k vs Normal")
ax.get_lines()[0].set(color="darkred", markersize=3, alpha=0.5)
ax.get_lines()[1].set(color="red", linewidth=2)

fig.suptitle("QQ Plots -- Departure from Normality in Upper Tail", fontsize=14, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()

# Formal normality tests (Shapiro on subsample for speed, D'Agostino on full data)
_, dag_p_mp = stats.normaltest(mp_rate)
_, dag_p_bd = stats.normaltest(bd_rate)
print("D'Agostino-Pearson normality test:")
print(f"  missing_per_100k  p-value = {dag_p_mp:.2e}  {'REJECT normality' if dag_p_mp < 0.05 else 'Fail to reject'}")
print(f"  bodies_per_100k   p-value = {dag_p_bd:.2e}  {'REJECT normality' if dag_p_bd < 0.05 else 'Fail to reject'}")
print("\nBoth distributions strongly violate normality, justifying robust methods.")

## 3. Standard vs. Robust Z-Scores

Standard z-scores use the mean and standard deviation, which are both heavily influenced by
outliers. Robust z-scores replace these with the **median** and **MAD (Median Absolute
Deviation)**, producing scores that are far less distorted by extreme values.

The key question: when we switch to robust z-scores, do some counties lose their "outlier"
status, and do others gain it?

In [ ]:
# --- Scatter: standard z vs robust z ---
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Missing persons
ax = axes[0]
valid = df.dropna(subset=["mp_z_score", "mp_robust_z"])
sc = ax.scatter(valid["mp_z_score"], valid["mp_robust_z"],
                c=valid["alert_level"].map({"GREEN": "green", "YELLOW": "gold", "RED": "red"}).fillna("grey"),
                s=8, alpha=0.4, edgecolors="none")
lim = max(abs(valid["mp_z_score"]).max(), abs(valid["mp_robust_z"]).max()) * 0.5
ax.plot([0, lim], [0, lim], "k--", alpha=0.4, label="y = x (identical scores)")
ax.set_xlabel("Standard Z-score (mp_z_score)")
ax.set_ylabel("Robust Z-score (mp_robust_z)")
ax.set_title("Missing Persons: Standard vs Robust Z")
ax.legend(loc="upper left")

# Bodies
ax = axes[1]
valid_b = df.dropna(subset=["bodies_z_score", "bodies_robust_z"])
sc = ax.scatter(valid_b["bodies_z_score"], valid_b["bodies_robust_z"],
                c=valid_b["alert_level"].map({"GREEN": "green", "YELLOW": "gold", "RED": "red"}).fillna("grey"),
                s=8, alpha=0.4, edgecolors="none")
lim_b = max(abs(valid_b["bodies_z_score"]).max(), abs(valid_b["bodies_robust_z"]).max()) * 0.5
ax.plot([0, lim_b], [0, lim_b], "k--", alpha=0.4, label="y = x (identical scores)")
ax.set_xlabel("Standard Z-score (bodies_z_score)")
ax.set_ylabel("Robust Z-score (bodies_robust_z)")
ax.set_title("Unidentified Bodies: Standard vs Robust Z")
ax.legend(loc="upper left")

plt.suptitle("Robust Z-scores amplify separation -- points above the diagonal are flagged more aggressively",
             fontsize=12, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# --- Count how many counties change alert level under robust scoring ---
# Define a simple threshold-based alert using robust z-scores
# Standard thresholds: GREEN < 2, YELLOW 2-3, RED >= 3

def assign_alert(z):
    if pd.isna(z):
        return "GREEN"
    if z >= 3:
        return "RED"
    elif z >= 2:
        return "YELLOW"
    return "GREEN"

df["alert_standard"] = df["mp_z_score"].apply(assign_alert)
df["alert_robust"] = df["mp_robust_z"].apply(assign_alert)

# Crosstab to show transitions
ct = pd.crosstab(df["alert_standard"], df["alert_robust"],
                 rownames=["Standard Alert"], colnames=["Robust Alert"])
print("Alert Level Transition Matrix (Standard -> Robust):\n")
print(ct)

# Summary counts
changed = (df["alert_standard"] != df["alert_robust"]).sum()
upgraded = ((df["alert_standard"] == "GREEN") & (df["alert_robust"].isin(["YELLOW", "RED"]))).sum()
downgraded = ((df["alert_standard"].isin(["YELLOW", "RED"])) & (df["alert_robust"] == "GREEN")).sum()

print(f"\nTotal counties that changed alert level: {changed:,}")
print(f"  Upgraded (GREEN -> YELLOW/RED under robust):  {upgraded:,}")
print(f"  Downgraded (YELLOW/RED -> GREEN under robust): {downgraded:,}")
print(f"\nRobust z-scores typically flag MORE counties because the MAD denominator is smaller than the SD.")

In [ ]:
# --- Interactive Plotly scatter for detailed exploration ---
sample = df.dropna(subset=["mp_z_score", "mp_robust_z"]).copy()
sample["label"] = sample["County"] + ", " + sample["State"] + " (" + sample["decade"].astype(str) + "s)"

fig = px.scatter(
    sample,
    x="mp_z_score",
    y="mp_robust_z",
    color="alert_level",
    color_discrete_map={"GREEN": "#2ca02c", "YELLOW": "#f0c929", "RED": "#d62728"},
    hover_name="label",
    hover_data={"missing_per_100k": ":.1f", "population": ":,.0f",
                "mp_z_score": ":.2f", "mp_robust_z": ":.2f"},
    opacity=0.5,
    title="Standard vs Robust Z-Scores -- Missing Persons (hover for details)",
    labels={"mp_z_score": "Standard Z-score", "mp_robust_z": "Robust Z-score (MAD)"},
)
fig.add_shape(type="line", x0=0, y0=0, x1=sample["mp_z_score"].quantile(0.99),
              y1=sample["mp_z_score"].quantile(0.99),
              line=dict(dash="dash", color="black", width=1))
fig.update_layout(height=550, template="plotly_white")
fig.show()

## 4. FDR Correction

When testing thousands of counties simultaneously, we expect many false positives by chance
alone. The **Benjamini-Hochberg FDR correction** controls the expected proportion of false
discoveries. Below we compare uncorrected p-values against FDR-adjusted significance.

In [ ]:
# --- Uncorrected vs FDR-corrected significance ---
valid_p = df.dropna(subset=["mp_p_value"]).copy()

uncorrected_sig = (valid_p["mp_p_value"] < 0.05).sum()
fdr_sig = valid_p["fdr_significant"].sum()

print("Significance comparison (alpha = 0.05):")
print(f"  Uncorrected (mp_p_value < 0.05):  {uncorrected_sig:,} counties ({uncorrected_sig/len(valid_p)*100:.1f}%)")
print(f"  FDR-corrected (fdr_significant):   {fdr_sig:,} counties ({fdr_sig/len(valid_p)*100:.1f}%)")
print(f"  Reduction:                         {uncorrected_sig - fdr_sig:,} fewer flagged ({(uncorrected_sig - fdr_sig)/max(uncorrected_sig,1)*100:.1f}%)")
print(f"\nTotal county-decades tested: {len(valid_p):,}")

In [ ]:
# --- Bar chart: Uncorrected vs FDR-corrected ---
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar comparison
ax = axes[0]
categories = ["Uncorrected\n(p < 0.05)", "FDR-Corrected\n(adjusted p < 0.05)"]
counts = [uncorrected_sig, fdr_sig]
colors = ["#e74c3c", "#2ecc71"]
bars = ax.bar(categories, counts, color=colors, edgecolor="black", linewidth=0.8, width=0.5)
for bar, count in zip(bars, counts):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 20,
            f"{count:,}", ha="center", va="bottom", fontweight="bold", fontsize=13)
ax.set_ylabel("Number of Significant County-Decades")
ax.set_title("Impact of FDR Correction on Significance Calls")

# P-value distribution (histogram)
ax = axes[1]
ax.hist(valid_p["mp_p_value"], bins=50, color="steelblue", edgecolor="white", alpha=0.85)
ax.axvline(0.05, color="red", ls="--", lw=2, label="alpha = 0.05")
ax.set_xlabel("Uncorrected p-value (mp_p_value)")
ax.set_ylabel("Frequency")
ax.set_title("Distribution of Uncorrected P-Values")
ax.legend()

plt.tight_layout()
plt.show()

print("Note: A uniform p-value distribution under the null would appear as a flat histogram.")
print("Excess density near zero indicates many genuinely anomalous counties.")

## 5. Empirical Bayes Shrinkage

Small counties with tiny populations produce wildly unstable rates. A single missing person
in a county of 500 people yields 200 per 100k -- an artifact of small denominators, not a
genuine signal.

**Empirical Bayes shrinkage** pulls extreme small-county rates toward the global mean,
weighted by `shrinkage_weight` (0 = fully shrunk to global mean, 1 = keep original rate).
Counties with large populations retain their original rate; small counties are stabilized.

In [ ]:
# --- Original rate vs shrunk rate, colored by shrinkage weight ---
shrink_df = df.dropna(subset=["missing_per_100k", "mp_rate_shrunk", "shrinkage_weight"]).copy()

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Missing persons
ax = axes[0]
sc = ax.scatter(shrink_df["missing_per_100k"], shrink_df["mp_rate_shrunk"],
                c=shrink_df["shrinkage_weight"], cmap="RdYlGn", s=10, alpha=0.6,
                edgecolors="none", vmin=0, vmax=1)
plt.colorbar(sc, ax=ax, label="Shrinkage Weight (0=full shrink, 1=keep original)")
max_val = max(shrink_df["missing_per_100k"].quantile(0.99), shrink_df["mp_rate_shrunk"].quantile(0.99))
ax.plot([0, max_val], [0, max_val], "k--", alpha=0.4, label="No shrinkage (y=x)")
ax.set_xlabel("Original Rate (missing_per_100k)")
ax.set_ylabel("Shrunk Rate (mp_rate_shrunk)")
ax.set_title("Missing Persons: Empirical Bayes Shrinkage")
ax.legend(loc="upper left")

# Bodies
shrink_bd = df.dropna(subset=["bodies_per_100k", "bodies_rate_shrunk", "shrinkage_weight"]).copy()
ax = axes[1]
sc2 = ax.scatter(shrink_bd["bodies_per_100k"], shrink_bd["bodies_rate_shrunk"],
                 c=shrink_bd["shrinkage_weight"], cmap="RdYlGn", s=10, alpha=0.6,
                 edgecolors="none", vmin=0, vmax=1)
plt.colorbar(sc2, ax=ax, label="Shrinkage Weight (0=full shrink, 1=keep original)")
max_val_b = max(shrink_bd["bodies_per_100k"].quantile(0.99), shrink_bd["bodies_rate_shrunk"].quantile(0.99))
ax.plot([0, max_val_b], [0, max_val_b], "k--", alpha=0.4, label="No shrinkage (y=x)")
ax.set_xlabel("Original Rate (bodies_per_100k)")
ax.set_ylabel("Shrunk Rate (bodies_rate_shrunk)")
ax.set_title("Unidentified Bodies: Empirical Bayes Shrinkage")
ax.legend(loc="upper left")

plt.suptitle("Points below the diagonal have been shrunk toward the global mean", fontsize=12, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# --- Extreme examples: small counties shrunk dramatically ---
extreme = shrink_df.copy()
extreme["shrinkage_delta"] = extreme["missing_per_100k"] - extreme["mp_rate_shrunk"]
extreme["shrinkage_pct"] = (extreme["shrinkage_delta"] / extreme["missing_per_100k"].replace(0, np.nan) * 100)

# Top 15 most shrunk counties
top_shrunk = extreme.nlargest(15, "shrinkage_delta")[
    ["State", "County", "decade", "population", "missing_per_100k",
     "mp_rate_shrunk", "shrinkage_delta", "shrinkage_weight", "small_county_flag"]
].reset_index(drop=True)

print("Top 15 Counties with Greatest Empirical Bayes Shrinkage (Missing Persons):")
print("These are predominantly small counties where raw rates are unreliable.\n")
print(top_shrunk.to_string(index=False))

print(f"\nSmall county flag prevalence in top 15: "
      f"{top_shrunk['small_county_flag'].sum()}/{len(top_shrunk)} "
      f"({top_shrunk['small_county_flag'].mean()*100:.0f}%)")
print(f"Mean population in top 15: {top_shrunk['population'].mean():,.0f}")
print(f"Mean shrinkage weight:     {top_shrunk['shrinkage_weight'].mean():.3f} (near 0 = heavy shrinkage)")

In [ ]:
# --- Interactive Plotly: shrinkage by population ---
fig = px.scatter(
    shrink_df,
    x="population",
    y="shrinkage_weight",
    color="small_county_flag",
    color_discrete_map={True: "#e74c3c", False: "#3498db"},
    hover_data={"County": True, "State": True, "decade": True,
                "missing_per_100k": ":.1f", "mp_rate_shrunk": ":.1f"},
    opacity=0.4,
    log_x=True,
    title="Shrinkage Weight vs Population -- Small counties receive more shrinkage",
    labels={"population": "Population (log scale)", "shrinkage_weight": "Shrinkage Weight",
            "small_county_flag": "Small County"},
)
fig.update_layout(height=450, template="plotly_white")
fig.show()

## 6. Method Comparison

Five different statistical methods are available for identifying outlier counties. Each makes
different assumptions about the data-generating process:

| Method | Column | Assumption |
|---|---|---|
| Standard Z-score | `mp_z_score` | Normal distribution of rates |
| Log-transformed Z | `mp_log_z` | Log-normal distribution |
| Poisson p-value | `mp_poisson_p` | Counts follow Poisson process |
| Negative Binomial p-value | `mp_nb_p` | Overdispersed counts |
| Percentile rank | `mp_percentile` | Non-parametric; no distributional assumption |

Below we count how many county-decades each method flags at the **99th percentile** threshold.

In [ ]:
# --- Outlier counts at 99th percentile for each method ---
methods = {
    "Standard Z >= 2.33":       (df["mp_z_score"] >= 2.33).sum(),
    "Log Z >= 2.33":            (df["mp_log_z"] >= 2.33).sum(),
    "Poisson p < 0.01":         (df["mp_poisson_p"] < 0.01).sum(),
    "Neg. Binomial p < 0.01":   (df["mp_nb_p"] < 0.01).sum(),
    "Percentile >= 99":         (df["mp_percentile"] >= 99).sum(),
}

method_df = pd.DataFrame({
    "Method": list(methods.keys()),
    "Outliers Flagged": list(methods.values()),
    "Pct of Total": [v / len(df) * 100 for v in methods.values()],
})
method_df = method_df.sort_values("Outliers Flagged", ascending=False).reset_index(drop=True)

print("Outlier Counts at 99th Percentile Threshold:\n")
print(method_df.to_string(index=False))
print(f"\nTotal county-decades: {len(df):,}")

In [ ]:
# --- Bar chart comparison of methods ---
fig, ax = plt.subplots(figsize=(12, 5))

colors = ["#2c3e50", "#2980b9", "#e67e22", "#e74c3c", "#27ae60"]
bars = ax.barh(method_df["Method"], method_df["Outliers Flagged"],
               color=colors, edgecolor="black", linewidth=0.6)

for bar, val in zip(bars, method_df["Outliers Flagged"]):
    ax.text(bar.get_width() + 5, bar.get_y() + bar.get_height() / 2,
            f"{val:,}", va="center", fontweight="bold")

ax.set_xlabel("Number of County-Decades Flagged as Outliers")
ax.set_title("Method Comparison: Outlier Detection at 99th Percentile", fontweight="bold")
ax.invert_yaxis()
plt.tight_layout()
plt.show()

print("Interpretation:")
print("- Parametric methods (Poisson, NB) may flag more or fewer counties depending on overdispersion.")
print("- The percentile method is non-parametric and flags exactly ~1% by construction.")
print("- Standard Z tends to be liberal (inflated by skew); Log Z is more conservative.")

In [ ]:
# --- Agreement heatmap: how many counties are flagged by multiple methods ---
df["flag_std_z"]     = (df["mp_z_score"] >= 2.33).astype(int)
df["flag_log_z"]     = (df["mp_log_z"] >= 2.33).astype(int)
df["flag_poisson"]   = (df["mp_poisson_p"] < 0.01).astype(int)
df["flag_nb"]        = (df["mp_nb_p"] < 0.01).astype(int)
df["flag_percentile"]= (df["mp_percentile"] >= 99).astype(int)

flag_cols = ["flag_std_z", "flag_log_z", "flag_poisson", "flag_nb", "flag_percentile"]
flag_labels = ["Standard Z", "Log Z", "Poisson", "Neg. Binom.", "Percentile"]

# Pairwise agreement (Jaccard-like overlap)
overlap = pd.DataFrame(index=flag_labels, columns=flag_labels, dtype=float)
for i, (ci, li) in enumerate(zip(flag_cols, flag_labels)):
    for j, (cj, lj) in enumerate(zip(flag_cols, flag_labels)):
        both = ((df[ci] == 1) & (df[cj] == 1)).sum()
        either = ((df[ci] == 1) | (df[cj] == 1)).sum()
        overlap.loc[li, lj] = both / either if either > 0 else 0

fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(overlap.values.astype(float), cmap="YlOrRd", vmin=0, vmax=1)
ax.set_xticks(range(len(flag_labels)))
ax.set_xticklabels(flag_labels, rotation=45, ha="right")
ax.set_yticks(range(len(flag_labels)))
ax.set_yticklabels(flag_labels)
for i in range(len(flag_labels)):
    for j in range(len(flag_labels)):
        val = float(overlap.iloc[i, j])
        ax.text(j, i, f"{val:.2f}", ha="center", va="center",
                color="white" if val > 0.6 else "black", fontsize=10)
plt.colorbar(im, ax=ax, label="Jaccard Overlap")
ax.set_title("Pairwise Agreement Between Outlier Detection Methods", fontweight="bold")
plt.tight_layout()
plt.show()

# How many methods agree for flagged counties
df["n_methods_flagged"] = df[flag_cols].sum(axis=1)
flagged_any = df[df["n_methods_flagged"] > 0]
print(f"\nCounty-decades flagged by at least one method: {len(flagged_any):,}")
print(f"Flagged by ALL 5 methods (high-confidence outliers): {(df['n_methods_flagged'] == 5).sum():,}")
print(f"\nDistribution of method agreement among flagged counties:")
print(flagged_any["n_methods_flagged"].value_counts().sort_index().to_string())

In [ ]:
# --- Interactive: top consensus outliers (flagged by all 5 methods) ---
consensus = df[df["n_methods_flagged"] == 5].copy()
consensus["label"] = consensus["County"] + ", " + consensus["State"]
consensus = consensus.sort_values("missing_per_100k", ascending=False)

fig = px.bar(
    consensus.head(30),
    x="label",
    y="missing_per_100k",
    color="decade",
    hover_data={"population": ":,.0f", "mp_z_score": ":.2f", "mp_robust_z": ":.2f",
                "mp_poisson_p": ":.2e", "mp_nb_p": ":.2e", "mp_percentile": ":.1f"},
    title="Top 30 Consensus Outliers -- Flagged by All 5 Methods (Missing Persons)",
    labels={"label": "County", "missing_per_100k": "Rate per 100k", "decade": "Decade"},
)
fig.update_layout(xaxis_tickangle=-45, height=500, template="plotly_white")
fig.show()

## 7. Summary of Key Findings

### Distribution Characteristics
- Both `missing_per_100k` and `bodies_per_100k` are **heavily right-skewed**, with
  skewness values well above 1.0. The D'Agostino-Pearson test formally rejects normality
  for both (p << 0.001). This means standard z-scores, which assume a symmetric bell curve,
  will systematically underestimate the extremity of the right tail.

### Robust Z-Scores
- Robust (MAD-based) z-scores produce **larger absolute values** for true outliers because
  the MAD denominator is not inflated by the very outliers it is trying to detect. Many
  counties that appeared moderate under standard z-scores are reclassified as YELLOW or RED
  under robust scoring.

### FDR Correction
- Without correction, a naive p < 0.05 threshold flags a large number of county-decades.
  After Benjamini-Hochberg FDR correction, the number of significant results drops
  substantially, removing likely false positives while retaining high-confidence anomalies.

### Empirical Bayes Shrinkage
- Small counties (low population) receive **heavy shrinkage** toward the global mean rate.
  This prevents tiny counties with 1-2 cases from dominating the outlier rankings.
  The most dramatically shrunk counties are almost exclusively those with populations
  under 10,000.

### Method Agreement
- Counties flagged by **all five methods** simultaneously represent the highest-confidence
  outliers. These consensus outliers are robust to the choice of distributional assumption
  and should be prioritized for further investigation.
- The Negative Binomial model, which accounts for overdispersion, tends to be the most
  conservative (fewest flags), while the Poisson model may be overly liberal due to
  unmodeled variance.

### Practical Recommendation
For downstream analysis, use a **tiered approach**:
1. **Tier 1 (highest confidence):** Counties flagged by all 5 methods AND surviving FDR correction.
2. **Tier 2 (high confidence):** Counties flagged by 3+ methods with shrunk rates still elevated.
3. **Tier 3 (investigate further):** Counties flagged by any single method, especially if large-population.